In [1]:
#essential imports
import pandas as pd
import numpy as np

import datetime as dt
import time

# Strict JSON conversion
import json 
import gzip 
 
startyear=1990
startmonth=1

cps = 4
# how many month before the death of the artist are used as feature 
nbMonthBeforeDeath = 4

datapath = 'DATA/review_filtered/'
f_sbj_reviews = datapath+'movie_reviews_matched_'+str(cps)+'cps.csv'
f_ctl_reviews = datapath+'movie_reviews_matched_'+str(cps)+'cps_ctrls.csv'

# Load data

reviews related to subject (=dead) artists

In [2]:
sbj_reviews = pd.read_csv(f_sbj_reviews).drop('Unnamed: 0',axis=1)
sbj_reviews = sbj_reviews.rename(columns={'reviewTime' : 'strReviewTime', 'actor name' : 'artist name', 
                                          'musician name' : 'artist name', 'name' : 'artist name'})
sbj_reviews.head(3)

,index,asin,strReviewTime,overall,helpful,summary,reviewText,actor name,death date
0,308659,0792840461,2006-04-07,4.0,"[1, 3]",The orginal,The Original and the best-not much to say read...,jack weston,1996-05-03
1,308660,0792840461,2013-03-24,5.0,"[0, 0]",Steve McQueen invented cool,This was a great movie Steve McQueen invented ...,jack weston,1996-05-03
2,308661,0792840461,2014-04-05,4.0,"[1, 1]",Where The Dough Is,"Everybody knows banks, whether in storefronts,...",jack weston,1996-05-03


reviews related to control (=dead) artists

In [3]:
ctl_reviews = pd.read_csv(f_ctl_reviews).drop('Unnamed: 0',axis=1)
ctl_reviews = ctl_reviews.rename(columns={'reviewTime' : 'strReviewTime', 'actor name' : 'artist name', 
                                          'musician name' : 'artist name', 'name' : 'artist name'}})
ctl_reviews.head(3)

,index,asin,strReviewTime,overall,helpful,summary,reviewText,actor name
0,827321,630327417X,2009-10-15,5.0,"[0, 1]",Great Movie,Nothing bad to say. Dvd came quickly and in gr...,michael hynson
1,827322,630327417X,2000-09-16,5.0,"[0, 0]",endless summer,its just to good for words!,michael hynson
2,827323,630327417X,2005-03-03,1.0,"[5, 58]",A very disappointed lover of documentaries,I feel obligated to warn you-as I wish I was w...,michael hynson


# Pre-process data 

- transform string dates into formats that are easier for us to work with
- keep on the side the 'death date' <-> 'artist name relation 

Convert dates to year and extract and set aside actor<-> deathdates

In [5]:
# returns as an int the month of the date passed as argument. Apply to a serie !
def strdateToMonth(date) : 
    year = int(str(date)[:4])
    month = int(str(date)[5:7])
    assert(month <= 12 and month >= 1)
    return (month - startmonth) + (year - startyear)*12 

# returns as an int the year of the date passed as argument. Apply to a serie !
def strdateToYear(date) : 
    return int(str(date)[:4])

# SUBJECTS
sbj_reviews['reviewMonth'] = sbj_reviews['strReviewTime'].apply(strdateToMonth)
sbj_reviews['reviewYear'] = sbj_reviews['strReviewTime'].apply(strdateToYear)
sbj_reviews['reviewTime'] = pd.to_datetime(sbj_reviews['strReviewTime'])
sbj_reviews['death month'] = sbj_reviews['death date'].apply(strdateToMonth)
sbj_reviews['death year'] = sbj_reviews['death date'].apply(strdateToYear)
sbj_reviews['death date'] = pd.to_datetime(sbj_reviews['death date'])

# CONTROLS
ctl_reviews['reviewMonth'] = ctl_reviews['strReviewTime'].apply(strdateToMonth)
ctl_reviews['reviewYear'] = ctl_reviews['strReviewTime'].apply(strdateToYear)
ctl_reviews['reviewTime'] = pd.to_datetime(ctl_reviews['strReviewTime'])

Set aside the 'death date' <-> 'artist name relation 

In [6]:
deathdates = subjects_actors.groupby(['actor name','death date', 'death year', 'death month']).count()\
                            .reset_index()[['actor name', 'death date', 'death year', 'death month']]
deathdates.head()

,actor name,death date,death year,death month
0,adriana caselotti,1997-01-19,1997,84
1,adrienne shelly,2006-11-01,2006,202
2,al martino,2009-10-13,2009,237
3,alain bashung,2009-03-14,2009,230
4,alan bates,2003-12-27,2003,167


Extract controls names

In [7]:
#TODO : remove
#ctrl_names = list(set(controls_actors['actor name']))\
#sbj_names = list(deathdates['actor name'])

# filtering

### 'mentionned' Boolean

We want to distinguish which artists are frequently mentionned in their reviews; i.e, those that are popular, as opposed to those that just appear to play/sing with some other, more popular artist.
We add the boolean ['mentionned'] that tells if the artist listed with the review is mentionnned in the text of the review.

In [8]:
thresh_pop = 1

def mentionsName(row) : 
    if type(row['reviewText']) is str :
        names = row['actor name'].split(' ')
        text =  row['reviewText'].lower()
        # can mention only name or surname
        #return row['actor name'] in row['reviewText']
        for name in names : 
            if name in text : 
                return True
        return False
    else : return False

In [10]:
## Treat subjects
##
sbj_reviews['mentionned'] = sbj_reviews.apply(mentionsName, axis=1)
sbj_reviews.shape, sbj_reviews[sbj_reviews['mentionned']==True].shape

((597276, 15), (130234, 15))

In [12]:
## Treat controls
##
ctl_reviews['mentionned'] = ctl_reviews.apply(mentionsName, axis=1)
ctl_reviews.shape, ctl_reviews[controls_actors['mentionned']==True].shape

((285071, 12), (42178, 12))

Not necessarily important for later : the list of the most-mentionned subjects and controls

In [ ]:
## Not necessarily needed, count of mentions are still in the reviews dataframe
##
sbj_popular = sbj_reviews[sbj_reviews['mentionned']==True]
sbj_popular = sbj_popular.groupby('actor name').count()['asin']
print(len(sbj_names), " reduced to", sbj_popular.shape)
sbj_popular.sort_values(ascending=False)[:5]

In [13]:
## Not necessarily needed, count of mentions are still in the reviews dataframe
##
ctl_popular = ctl_reviews[ctl_reviews['mentionned']==True]
ctl_popular = ctl_popular.groupby('actor name').count()['asin']
print(len(ctrl_names), " reduced to", ctl_popular.shape)
ctl_popular.sort_values(ascending=False)[:5]

2419  reduced to (1215,)


actor name
matt damon            6017
steve martin          3266
tyler perry           2870
ian anderson          1213
laurence fishburne    1198
liev schreiber        1060
michelle williams     1058
marc m                 896
maggie gyllenhaal      867
robert taylor          749
Name: asin, dtype: int64

### Date filtering

We're only interested in the reviews of subjects that died in a recent time period because amazon's popularity is recent, so we'll have much more amazon data after year 2000. Therefore, we will only keep the reviews for which the subject died within a 2003-2014 time period, otherwise, the artist died too long ago.

TODO : RE-CHECK THIS, THIS WAS 1996-2013 ORIGINALLY
TODO : the filtered variable was bound to 'subjects in time' and not reused later on

In [14]:
start_year = 2003
end_year = 2014

print('original size : ', sbj_reviews.shape)
sbj_reviews = sbj_reviews[(sbj_reviews['death year']>=start) \
                        & (sbj_reviews['death year']<=end)]
print('filtered size : ', sbj_reviews.shape)
sbj_reviews.head(3)

original size :  (597276, 15)
filtered size :  (597276, 15)


,index,asin,strReviewTime,overall,helpful,summary,reviewText,actor name,death date,reviewMonth,reviewYear,reviewTime,death month,death year,mentionned
0,308659,0792840461,2006-04-07,4.0,"[1, 3]",The orginal,The Original and the best-not much to say read...,jack weston,1996-05-03,195,2006,2006-04-07,76,1996,False
1,308660,0792840461,2013-03-24,5.0,"[0, 0]",Steve McQueen invented cool,This was a great movie Steve McQueen invented ...,jack weston,1996-05-03,278,2013,2013-03-24,76,1996,False
2,308661,0792840461,2014-04-05,4.0,"[1, 1]",Where The Dough Is,"Everybody knows banks, whether in storefronts,...",jack weston,1996-05-03,291,2014,2014-04-05,76,1996,False


# Extract features used to match controls with subjects

We will use as features the number of review per month for a 4 months before the death of the subject. We obtain a features vector that we will try to match best between control and subjects.

Since the controls didn't die however, and since the amount of amazon reviews is generated increasingly fast as time passses, we will this feature for every control, for every subject we try to match it to, using the subject's death date as reference point and using the control's number of reviews one month before, two months before, and so on.

In [15]:
# relative count of months before death (-1, -2, ...)
relMonthBeforeDeath = [-i for i in range(1, nbMonthBeforeDeath+1)]
# string matching the feature's column in the subject's dataframe
relFeaturesStr = { i : 'revs'+str(i) for i in relMonthBeforeDeath}

In [16]:
# SUBJECTS COUNTS BY MONTH
sbj_reviews_monthly = sbj_reviews.groupby(['actor name','reviewMonth']).count()


''' returns the number of reviews for the subject in row, 'relMonthNb' months before his death month
'''
def getreview_month_subjects(row, relMonthNb) : 
    actor = row['actor name']
    deathMonth = int(row['death month'])
    try : 
        val = sbj_reviews_monthly.loc[actor, int(deathMonth+relMonthNb)]['asin']
        return int(val)
    ## Both exceptions can happen when the indexing keys are 
    ## not in the index == when there were no review for the actor at the given month
    except TypeError :
        #print('key (', actor, ', '+str(deathMonth+monthNb)+') returned typeError')
        return 0
    except KeyError :
        #print('key (', actor, ', ', deathMonth, ') found no match in the index')
        return 0
    

    
sbj_monthly_feats = pd.DataFrame(deathdates[['actor name', 'death month']])
for i in relMonthBeforeDeath :
    sbj_monthly_feats[relFeaturesStr[i]] = sbj_monthly_feats\
                                    .apply(lambda row : getreview_month_subjects(row, i), axis=1)

## Compute the norm of the feature
##
def feature_norm(row) : 
    feature = [row[relFeaturesStr[i]] for i in relMonthBeforeDeath]
    return np.linalg.norm(feature)

sbj_monthly_feats['featureNorm'] = sbj_monthly_feats.apply(feature_norm, axis=1)
sbj_monthly_feats['death date'] = deathdates['death date']
        
sbj_monthly_feats.sort_values(by='featureNorm', ascending=False).head()

,actor name,death month,revs-1,revs-2,revs-3,revs-4,featureNorm,death date
357,mickey rooney,291,149,156,236,181,367.415296,2014-04-06
228,james garner,294,151,178,169,196,348.513988,2014-07-19
140,eli wallach,293,120,138,114,148,261.426854,2014-06-24
389,philip seymour hoffman,289,119,106,68,77,189.604852,2014-02-02
288,karen black,283,74,111,121,39,184.279679,2013-08-08


In [17]:
# SUBJECTS COUNTS BY YEAR
subjects_revs = subjects_actors.groupby(['actor name','reviewYear']).count()


def getreview(actor, year) : 
    try : 
        val = subjects_revs.loc[actor, year]['asin']
        return val
    except TypeError :
        return 0
    
    

# FILTER TO GET THE DEATH YEAR COUNT - 1
subjects_feats = deathdates[1:].apply(lambda row : getreview(row['actor name'], int(row['death year'])-1), axis=1)
subjects_feats = pd.DataFrame(subjects_feats, columns=['reviews'])
subjects_feats['actor name'] = deathdates['actor name']
subjects_feats['death year'] = deathdates['death year']
subjects_feats['death date'] = deathdates['death date']

subjects_feats.sort_values(by='reviews', ascending=False).head()

,reviews,actor name,death year,death date
228,2054,james garner,2014,2014-07-19
389,1550,philip seymour hoffman,2014,2014-02-02
357,1425,mickey rooney,2014,2014-04-06
140,1112,eli wallach,2014,2014-06-24
51,1057,bob hoskins,2014,2014-04-29


Extract features for controls (?)

In [18]:
controls_revs = controls_actors.groupby(['actor name','reviewMonth'])\
                                .count()['asin']\
                                .to_frame().reset_index()

In [19]:
controls_revs.sort_values(by=['actor name', 'asin'], ascending=False).head()

,actor name,reviewMonth,asin
49759,zuzana parmova,143,1
49760,zuzana parmova,168,1
49743,zuhal gencer,185,2
49740,zuhal gencer,180,1
49741,zuhal gencer,182,1


# Matching

Prepare the control's dataframe for matching, and create a function that computes the features for subjects

In [20]:
# copy back above
def getreview_month_controls(row, monthNb, musiciansDataframe, deathMonth=None) : 
    actor = row['actor name']
    if deathMonth==None : 
        deathMonth = int(row['death month'])
    try : 
        val = musiciansDataframe.loc[actor, int(deathMonth+monthNb)]['asin']
        return int(val)
    ## Both exceptions can happen when the indexing keys are 
    ## not in the index == when there were no review for the actor at the given month
    except TypeError :
        #print('key (', actor, ', '+str(deathMonth+monthNb)+') returned typeError')
        return 0
    except KeyError :
        #print('key (', actor, ', ', deathMonth, ') found no match in the index')
        return 0

In [21]:
subjects_monthly_feats.head()
controls_revs.set_index(['actor name','reviewMonth']).loc['adrian holmes']
controls_revs.groupby('actor name').count().reset_index().head()

,actor name,reviewMonth,asin
0,aaron chayrez,1,1
1,aaron kwok,60,60
2,aarti chopra,2,2
3,abdul salaam el razzac,5,5
4,adam clark,19,19


In [22]:
def unwrap(subjectStuff) : 
    return subjectStuff.tolist()[0]

def match(actorname, threshold, ctrl_revs) : 

    # extract the useful features for the subject 
    sbj_actor = subjects_monthly_feats[subjects_monthly_feats['actor name'] == actorname]
    deathmonth = int(sbj_actor['death month'])
    sbjname = unwrap(sbj_actor['actor name'])
    feature = [unwrap(sbj_actor[relFeaturesStr[i]]) for i in relMonthBeforeDeath]
    
    #print(sbj_actor)
    #print('feature : ', feature)
    
    for control in ctrl_revs.groupby('actor name').count().reset_index().iterrows() : 
        
        # iterrows returns tuple (0, row)
        idx = control[0]
        control = control[1]
        
        #if idx % 100 == 0 : 
        #    print('    iterated over ', idx, 'controls (out of ', 
        #          ctrl_revs.groupby('actor name').count().reset_index().shape[0], ')')
        
        # Compute feature
        ctrl_feature = [getreview_month_controls(control, i, 
                                        ctrl_revs.set_index(['actor name', 'reviewMonth']), 
                                        deathMonth=deathmonth) \
                        for i in relMonthBeforeDeath]
        
        #print('   ctrl_feature : ', ctrl_feature )
        
        diff = np.linalg.norm(np.array(feature) - np.array(ctrl_feature))
        
        #print('    diff : ', diff)
        
        if diff <= threshold : 
            matched_name = control['actor name']
            matched_count = ctrl_feature
            
            #ctrl_revs = ctrl_revs[ctrl_revs['actor name'].isin([matched_name])==False]
            ctrl_revs = ctrl_revs[ctrl_revs['actor name']!=matched_name]
            
            print('    ', sbjname, ' matched with ', matched_name)
            print('    ', feature, ' ~ ' , ctrl_feature)
            return matched_name, ctrl_feature, ctrl_revs
    
    print('1 iteration completed over all controls')
    return 0, 0, ctrl_revs  


In [ ]:
ctrl_revs = controls_revs.copy()
print(len(ctrl_names), len(sbj_names))

thresh = 10
ctrl_revs = controls_revs.copy()
sorted_subjects = subjects_monthly_feats.sort_values(by='featureNorm', ascending=False)

sbj_list = []
sbj_feat_list = []
sbj_death_list = []
ctl_list = []
ctl_feat_list = []

for idx, subject_name in enumerate(sorted_subjects['actor name']):
    
    if idx%50 == 0: 
        print(idx, " subjects out of ", len(subjects_feats['actor name']))
    
    # extract the feature for the subject and use it to set the threshold
    sbj_actor = sorted_subjects[sorted_subjects['actor name'] == subject_name]
    feature = [unwrap(sbj_actor[relFeaturesStr[i]]) for i in relMonthBeforeDeath]
    
    sbjname = unwrap(sbj_actor['actor name'])
    print(sbjname, '-', feature)
    
    featureNorm = sbj_actor['featureNorm']
    thr = int(8 + 0.12*featureNorm)
    mtch = 0

    while mtch == 0:
        
        thr += int(0.3*thr + 1)
        print('  thr : ', thr)
        
        if thr > 50 and thr < 60  : 
            print(subject_name+' reached threshold ', thr)
        if thr > 1000 and thr < 1250  : 
            print(subject_name+' reached threshold ', thr)
        
        name, ctrl_feature, ctrl_revs_ = match(subject_name,thr,ctrl_revs)
        ctrl_revs = ctrl_revs_.sample(frac=1) 
        
        
        if name != 0:
            
            #print(subject_name+' matched with '+name)
            
            sbjDeathDate = unwrap(sbj_actor['death date'])
            
            sbj_list.append(sbjname)
            sbj_feat_list.append(feature)
            sbj_death_list.append(sbjDeathDate)
            ctl_list.append(name)
            ctl_feat_list.append(ctrl_feature)
            
            mtch += 1
            
print('DONE')

2419 501
0  subjects out of  500
mickey rooney - [149, 156, 236, 181]
  thr :  79
     mickey rooney  matched with  liev schreiber
     [149, 156, 236, 181]  ~  [158, 201, 257, 241]
james garner - [151, 178, 169, 196]
  thr :  74
     james garner  matched with  laurence fishburne
     [151, 178, 169, 196]  ~  [144, 155, 144, 208]
eli wallach - [120, 138, 114, 148]
  thr :  59
eli wallach reached threshold  59
     eli wallach  matched with  ben miles
     [120, 138, 114, 148]  ~  [94, 102, 120, 138]
philip seymour hoffman - [119, 106, 68, 77]
  thr :  46
     philip seymour hoffman  matched with  cheech marin
     [119, 106, 68, 77]  ~  [100, 95, 43, 54]
karen black - [74, 111, 121, 39]
  thr :  46
1 iteration completed over all controls
  thr :  70
     karen black  matched with  albert brooks
     [74, 111, 121, 39]  ~  [91, 98, 72, 76]
bob hoskins - [97, 88, 95, 87]
  thr :  46
     bob hoskins  matched with  maggie gyllenhaal
     [97, 88, 95, 87]  ~  [90, 71, 90, 50]
julie harris

# Results

In [ ]:
res = pd.DataFrame()

res['sbj'] = sbj_list
res['sbj_feats'] = sbj_feat_list
res['death date'] = sbj_death_list
res['ctl_feats'] = ctl_feat_list
res['ctl'] = ctl_list

res.head()

In [ ]:
ctrl_features = pd.DataFrame(ctrl_list, columns=['ctrl_name'])
ctrl_features['ctrl_reviews'] = count_list

res = pd.concat([ctrl_features.reset_index(), sorted_subjects.reset_index()], axis=1)\
            .drop('index', axis=1)
res['error'] = np.abs(res['ctrl_reviews'] - res['reviews'])/(2*(1+res['ctrl_reviews'] + res['reviews']))

res.sort_values(by='error', ascending=False).head()

In [ ]:
ctrl_features.hist(column='ctrl_reviews', figsize=[10,10], bins=100)

In [ ]:
subjects_feats.hist(column='reviews', figsize=[10,10], bins=100)

# Export results

In [ ]:
subjectsnb = str(len(sbj_names)-1)

In [ ]:
subjects_actors.to_csv('reviews_subjects_.csv', index=False)

In [ ]:
controls_actors.to_csv('reviews_controls_.csv', index=False)

In [ ]:
res.to_csv('DATA/reviews_cs_matched/matchings_'+subjectsnb+'_feature=yearbefore.csv', index=False)